Autor: Diego Arturo Padilla Domínguez
Matricula: A01552594
Fecha de entrega:19/11/2021

Librerias

In [39]:
import numpy as np
import pandas as pd
import random
import mesa
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128
import time
import datetime

Cración modelo

In [45]:
def crear_Matriz(modelo):
    matriz = np.zeros((modelo.grid.width, modelo.grid.height))
    for casilla in modelo.grid.coord_iter():
        cont_cas, x, y = casilla
        for cont in cont_cas:
            if isinstance(cont, Aspiradora):
                matriz[x][y] = 2
            else:
                matriz[x][y] = cont.est
    return matriz
    
class Aspiradora(Agent):
    
    def __init__(self, id_unico, modelo):
        super().__init__(id_unico, modelo)
        self.n_pos = None
        self.movs = 0
        
    def step(self):
        adys = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=True)
        
        for ady in adys:
            if isinstance(ady, Piso) and ady.pos == self.pos:
                ady.nEst = ady.est
                if ady.nEst == ady.s: 
                    ady.nEst = ady.l
                    self.n_pos = self.pos
                else:
                    vecinos = self.model.grid.get_neighborhood(
                        self.pos,
                        moore=True,
                        include_center=False)
                    n_pos = self.random.choice(vecinos)
                    self.n_pos = n_pos
                break
        
    def advance(self):
        
        adys = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=True)
        
        for ady in adys:
            if isinstance(ady, Piso) and ady.pos == self.pos: 
                ady.est = ady.nEst
                break
        
        if self.pos != self.n_pos:
            self.movs = self.movs + 1
            
        self.model.grid.move_agent(self, self.n_pos)
    
class Piso(Agent):
    
    s = 1
    l = 0
    
    def __init__(self, pos, modelo, est=l):
        super().__init__(pos, modelo)
        self.x, self.y = pos
        self.est = est
        self.nEst = None

class Matriz(Model):
    
    def __init__(self, m, n, nAspiradoras, pCeldasS):
        self.nAspiradoras = nAspiradoras
        self.pCeldasS = pCeldasS
        self.pCeldasL = 1 - pCeldasS
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)
               
        celdasS = int((m * n) * pCeldasS)
        listVacias = list(self.grid.empties)
        for celdas in range(celdasS):
            celda_vacia = random.choice(listVacias)
            piso = Piso(celda_vacia, self)
            piso.est = piso.s
            self.grid.place_agent(piso, celda_vacia)
            self.schedule.add(piso)
            listVacias.remove(celda_vacia)
        
        listVacias = list(self.grid.empties)
        for celdas in listVacias:
            piso = Piso(celdas, self)
            self.grid.place_agent(piso, celdas)
            self.schedule.add(piso)
        
        for i in range(nAspiradoras):
            aspiradora = Aspiradora(i, self)
            self.grid.place_agent(aspiradora, (1,1))
            self.schedule.add(aspiradora)
            
        self.dataC = DataCollector(
            model_reporters={'Matriz': crear_Matriz},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'movs', None)}
        )
    
    def step(self):
        self.dataC.collect(self)
        self.schedule.step()    
    
    def todasceldaslimpias(self):
        celdasL = 0
        for casilla in self.grid.coord_iter():
            cont_cas, x, y = casilla
            for cont in cont_cas:
                if isinstance(cont, Piso) and cont.est == cont.l:
                    celdasL = celdasL + 1
        
        self.pCeldasL = celdasL / (self.grid.width * self.grid.height)
        if self.pCeldasL == 1:
            return True
        else:
            return False

Declaración y ejecución del modelo

In [76]:
x = 10 #Tamaño matriz x
y = 10 #Tamañp matriz y

nAgentes = 80 #Cantidad de aspiradoras

PCeldasSucias = 0.5 #Porcentaje de casillas sucias

tMax = 0.5 #Tiempo maximo para limpiar

start = time.time()
tI = str(datetime.timedelta(seconds=tMax))
modelo = Matriz(x, y, nAgentes, PCeldasSucias)

while((time.time() - start) < tMax and not modelo.todasceldaslimpias()):
    modelo.step()

tF = str(datetime.timedelta(seconds=(time.time() - start)))

Grafiación

In [77]:
complete = modelo.dataC.get_model_vars_dataframe()

In [78]:
%%capture

fig, axs = plt.subplots(figsize=(5,5))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(complete.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(complete.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(complete))

In [79]:
anim

Informe

In [80]:
movs = modelo.dataC.get_agent_vars_dataframe()

print('Tiempo necesario para que todas las celdas estén limpias:', tF)
por = modelo.pCeldasL
print('Porcentaje de celdas limpias después del termino de la simulación:', por*100, "%")
print('Número de movimientos realizados por todos los agentes:', movs.tail()['Movimientos'].sum())

Tiempo necesario para que todas las celdas estén limpias: 0:00:00.067854
Porcentaje de celdas limpias después del termino de la simulación: 100.0 %
Número de movimientos realizados por todos los agentes: 68.0


La cantidad de agentes ayuda a que termine el proceso mas rapido, pero esto depende de la cantidad de celdas, esto debido a que el cambio en la cantidad del tiempo se ve mas afectado cuando hay una matriz mas grande que se tiene que recorrer la aleatoreidad con mas agentes hace que se mas facil que en algun momento pasen por todas las celdas en una menor cantidad de tiempo, por ende tambien se reduce la cantidad de movimientos.